# Semantic Kernel  & Agent-to-Agent (A2A) Protocol

이 노트북에서는 Semantic Kernel을 사용해 **Agent-to-Agent(A2A) 통신**을 구현하는 방법을 보여줍니다.  
우리는 두 개의 에이전트를 생성하게 됩니다:

1. **Flight Booking Agent** – 항공권 예약을 담당하는 전문 에이전트  
2. **Travel Planning Agent** – 여행 전반을 담당하며, Flight Booking Agent를 tool처럼 활용하는 에이전트  

## What does Agent-to-Agent (A2A) Protocol offer?

A2A Protocol은 서로 다른 AI 에이전트들이 **서로의 서비스를 호출하며 협업**할 수 있도록 해줍니다.  
이를 통해 다음과 같은 분산 시스템을 구성할 수 있습니다:

- 각 에이전트는 특정 작업에 전문화될 수 있다  
- 에이전트는 다른 에이전트의 기능을 활용할 수 있다  
- 전체 시스템이 더 모듈형(modular)·확장 가능(scalable)해진다  

## Architecture Overview

```
User Request → Travel Planning Agent → Flight Booking Agent → Response
```


Travel Planning Agent는 오케스트레이터 역할을 하며 다양한 여행 관련 요청을 처리합니다.  
이 중 **항공권 예약이 필요할 때**, A2A 프로토콜을 통해 전문 에이전트인 Flight Booking Agent와 통신합니다.

## Prerequisites and Setup

먼저, 필요한 dependencies를 설치하고 환경을 설정해봅시다.

In [ ]:
# Install required packages
# If you already did the uv configuration, you can skip this step
#%pip install semantic-kernel python-dotenv fastapi uvicorn httpx a2a-sdk

In [ ]:
# Import necessary libraries
import os
import logging
import threading
from uuid import uuid4

# Third-party imports
import httpx
import uvicorn
from dotenv import load_dotenv

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Load environment variables
load_dotenv('../.env')

print("✅ Dependencies installed and environment configured!")


## Flight Booking Agent Implementation

Flight Booking Agent는 **항공권 예약 요청**을 처리하는 데 특화된 에이전트입니다.  
사용자의 요청을 이해하고, 대화 문맥을 유지하며 흐름에 따라 필요한 정보를 수집합니다.

### Understanding the Flight Booking Agent Architecture

구현에 들어가기 전에, 이 에이전트가 어떤 점에서 특별한지 살펴봅시다:

1. **Semantic Kernel Integration**  
   Semantic Kernel의 `ChatCompletionAgent`를 사용해 에이전트를 인스턴스화합니다.

2. **Context Management**  
   각 대화는 별도의 context ID로 구분되어 관리되며,  
   이렇게 하면 여러 사용자 또는 세션을 동시에 처리할 수 있습니다.

3. **Conversation History**  
   에이전트는 context별로 과거 대화 내용을 기억하며  
   multi-turn conversation을 유지합니다.

이 에이전트는 다음과 같은 **stateful conversation pattern**을 따릅니다:

- 항공권 예약에 필요한 정보를 단계적으로 수집한다  
- 여러 턴(turn)을 거쳐도 context를 유지한다  
- 사용자에게 확인(confirmation) 및 요약(summary)을 제공한다

In [ ]:
# Import Semantic Kernel components
from semantic_kernel.agents.chat_completion.chat_completion_agent import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents.chat_history import ChatHistory

class SemanticKernelFlightBookingAgent:
    """A flight booking agent using Semantic Kernel and Azure OpenAI."""

    def __init__(self):
        """Initialize the flight booking agent with Azure OpenAI service."""
        logger.info("Initializing SemanticKernelFlightBookingAgent.")

        self.chat_agent = ChatCompletionAgent(
            service=AzureChatCompletion(),
            name="FlightBookingAssistant",
            instructions=(
                "You are a helpful flight booking assistant. "
                "Your task is to help users book flights by gathering necessary information "
                "such as departure city, destination city, travel dates, number of passengers, "
                "and preferred class of service. Once you have all the required information, "
                "provide a confirmation summary and simulate a successful booking."
            )
        )

        # Store chat history per context to maintain conversation state
        self.history_store: dict[str, ChatHistory] = {}

        logger.info("SemanticKernelFlightBookingAgent initialized successfully.")

    def _get_or_create_chat_history(self, context_id: str) -> ChatHistory:
        """Get existing chat history or create a new one for the given context."""
        chat_history = self.history_store.get(context_id)

        if chat_history is None:
            chat_history = ChatHistory(
                messages=[],
                system_message=(
                    "You are a helpful flight booking assistant. "
                    "Help users book flights by gathering all necessary information: "
                    "departure city, destination city, travel dates, number of passengers, "
                    "and preferred class. Once you have complete information, "
                    "provide a booking confirmation summary."
                )
            )
            self.history_store[context_id] = chat_history
            logger.info(f"Created new ChatHistory for context ID: {context_id}")

        return chat_history

    async def chat(self, user_input: str, context_id: str) -> str:
        """
        Process a flight booking request from the user.

        Args:
            user_input: The user's request for flight booking
            context_id: The context ID for maintaining conversation state

        Returns:
            The response from the flight booking agent
        """
        logger.info(f"Received flight booking request: {user_input} with context ID: {context_id}")

        if not user_input or not user_input.strip():
            logger.error("User input is empty.")
            raise ValueError("User input cannot be empty.")

        try:
            # Get or create chat history for the context
            chat_history = self._get_or_create_chat_history(context_id)

            # Add user input to chat history
            chat_history.messages.append(
                ChatMessageContent(role="user", content=user_input))

            # Create a new thread from the chat history
            thread = ChatHistoryAgentThread(
                chat_history=chat_history, thread_id=str(uuid4()))

            # Get response from the agent
            response = await self.chat_agent.get_response(message=user_input, thread=thread)

            # Add assistant response to chat history
            chat_history.messages.append(ChatMessageContent(
                role="assistant", content=response.content.content))


            logger.info(f"Chat history length is {len(chat_history.messages)} messages for context ID: {context_id}")
            logger.info(f"Flight booking agent response: {response.content.content}")

            return response.content.content

        except Exception as e:
            logger.error(f"Error processing flight booking request: {e}")
            return f"I apologize, but I encountered an error while processing your flight booking request: {str(e)}"

# Create an instance of the flight booking agent
flight_booking_agent = SemanticKernelFlightBookingAgent()
print("✅ Flight Booking Agent created successfully!")


### Test the Flight Booking Agent

이제 Flight Booking Agent가 어떻게 동작하는지 직접 테스트해 봅시다:

In [ ]:
response = await flight_booking_agent.chat(  
    user_input="I want to book a flight from New York to London on July 15th", 
    context_id="test_context_1"
)
print("-" * 50)
print("Final Response")
print("-" * 50)
print(f"Agent: {response}")
print("-" * 50)

### What Just Happened?

위 테스트에서 에이전트는 다음과 같은 방식으로 동작했습니다:

1. **Extracted Information**  
   사용자의 자연어 입력에서 출발지(New York), 도착지(London), 날짜(July 15th)를 정확히 추출했습니다.

2. **Identified Missing Information**  
   여전히 필요한 정보(승객 수, 좌석 클래스)를 파악했습니다.

3. **Maintained Context**  
   응답을 명확하고 구조적으로 정리해 conversation context를 유지했습니다.

4. **Guided the Conversation**  
   나머지 정보를 요청하며 자연스러운 방식으로 대화를 이끌었습니다.

이러한 행동을 통해 에이전트가 **부분적인 정보만 있어도 사용자에게 필요한 흐름을 안내하며 전체 예약 프로세스를 완성하도록 돕는 능력**을 갖추고 있음을 확인할 수 있습니다.

### 예약 흐름 완료 (Completing the Booking Flow)

이제 **빠진 정보**를 제공하면서 대화를 이어가 봅시다:

In [ ]:
response = await flight_booking_agent.chat(
    user_input="I need it for 2 passengers, business class, mock the rest",
    context_id="test_context_1"
)
print("-" * 50)
print("Final Response")
print("-" * 50)
print(f"Agent: {response}")
print("-" * 50)

### Conversation Continuity in Action

완벽합니다! 에이전트는 다음과 같이 동작했습니다:

1. **Remembered the Context**  
   동일한 대화 흐름에서 이전에 제공된 모든 정보를 기억했습니다.

2. **Processed New Information**  
   "2 passengers"와 "business class"라는 새로운 정보를 정확히 이해했습니다.

3. **Provided Confirmation**  
   지금까지 수집된 예약 정보를 깔끔하게 요약해 주었습니다.

4. **Simulated Success**  
   실제 거래처럼 예약이 성공적으로 완료되었다고 응답했습니다.

이는 동일한 context ID(`test_context_1`) 내에서  
**대화 히스토리가 유지되고 연결되는 방식**을 잘 보여줍니다.

---
### Testing Context Isolation

이제 **다른 context ID**를 사용해 테스트해 보면서,  
대화 context가 어떻게 **서로 분리(isolated)**되는지 확인해봅시다:

In [ ]:
response = await flight_booking_agent.chat(
        "Book a flight to Paris tomorrow", 
        "test_context_2"
    )
print("-" * 50)
print("Final Response")
print("-" * 50)
print(f"Agent: {response}")
print("-" * 50)

### Context Isolation Explained

`test_context_2`를 사용하자 완전히 새로운 대화가 시작된 것을 확인할 수 있습니다:

- **No Memory of Previous Booking**  
  에이전트는 이전의 New York → London 예약 정보를 전혀 기억하지 않습니다.

- **Fresh Start**  
  이 대화를 완전히 새로운 요청으로 인식하고, 필요한 모든 정보를 다시 요청합니다.

- **Independent State**  
  각 context는 **자기만의 대화 히스토리**를 독립적으로 유지합니다.

이러한 구조는 여러 사용자나 세션이 동시에 사용되는 상황에서  
각 사용자의 대화가 서로 **격리된 상태로 유지되어야 할 때** 매우 중요합니다.

---

### Understanding A2A Protocol Components

이제 작동하는 항공권 예약 에이전트를 만들었으니, A2A 프로토콜을 통해 다른 에이전트가 이 에이전트를 사용할 수 있도록 공개해야 합니다. 이를 위해서는 몇 가지 구성 요소가 필요합니다:

#### 1. **Agent Executor**
- A2A 프로토콜과 우리 에이전트 사이를 **브리지**합니다.
- A2A 요청을 에이전트 메서드 호출로 **변환**합니다.
- 응답을 위한 **이벤트 큐**를 관리합니다.

#### 2. **Agent Card**
- 에이전트의 **능력(capabilities)**을 표준화된 형식으로 설명합니다.
- 사용 가능한 **스킬(skills)**과 그 설명을 나열합니다.
- 에이전트를 **어떻게 사용하는지 예시**를 제공합니다.

#### 3. **A2A Server**
- 에이전트를 **HTTP 엔드포인트**로 노출합니다.
- A2A 프로토콜 기반의 **통신**을 처리합니다.
- **요청/응답** 흐름을 관리합니다.

이제 **Agent Executor**부터 단계별로 구현해 봅시다.

In [ ]:
# Import A2A SDK components
from a2a.server.agent_execution import AgentExecutor, RequestContext
from a2a.server.events import EventQueue
from a2a.utils import new_agent_text_message, new_task

class SemanticKernelFlightBookingAgentExecutor(AgentExecutor):
    """Executor for SemanticKernelFlightBookingAgent that handles A2A protocol integration."""

    def __init__(self):
        """Initialize the executor with a flight booking agent instance."""
        logger.info("Initializing SemanticKernelFlightBookingAgentExecutor.")
        self.agent = SemanticKernelFlightBookingAgent()
        logger.info("SemanticKernelFlightBookingAgentExecutor initialized successfully.")

    async def execute(
        self,
        context: RequestContext,
        event_queue: EventQueue,
    ) -> None:
        """
        Execute a flight booking request.

        Args:
            context: The request context containing user input and task information
            event_queue: Queue for sending events and responses
        """
        user_input = context.get_user_input()
        task = context.current_task
        context_id = context.context_id

        # Create a new task if one doesn't exist
        if not task:
            task = new_task(context.message)
            await event_queue.enqueue_event(task)

        logger.info(f"Executing flight booking - User input: {user_input}, Task ID: {task.id}, Context ID: {context_id}")

        try:
            # Process the flight booking request
            result = await self.agent.chat(user_input, context_id)

            # Send the result back through the event queue
            await event_queue.enqueue_event(new_agent_text_message(result))

            logger.info("Flight booking executed successfully.")

        except ValueError as ve:
            logger.error(f"Validation error during flight booking: {ve}")
            await event_queue.enqueue_event(
                new_agent_text_message(f"I need more information to help you book a flight: {str(ve)}")
            )

        except Exception as e:
            logger.error(f"Unexpected error during flight booking execution: {e}")
            await event_queue.enqueue_event(
                new_agent_text_message(
                    "I apologize, but I encountered an error while processing your flight booking request. "
                    "Please try again or contact support if the issue persists."
                )
            )

    async def cancel(
        self,
        context: RequestContext,
        event_queue: EventQueue
    ) -> None:
        """
        Handle cancellation requests.

        Args:
            context: The request context
            event_queue: Queue for sending events
        """
        logger.warning("Cancel operation requested but not supported for flight booking agent.")
        raise Exception('Cancel operation not supported for flight booking operations.')

print("✅ Agent Executor created successfully!")


### Agent Executor Explained

`SemanticKernelFlightBookingAgentExecutor` 클래스는 A2A 구조에서 매우 중요한 **브리지 역할**을 합니다:

1. **Protocol Translation**  
   A2A 프로토콜 요청을 에이전트의 `chat` 메서드 호출로 변환합니다.

2. **Context Management**  
   A2A 요청에서 context 정보를 추출해 올바른 대화 흐름을 유지합니다.

3. **Event Handling**  
   호출한 에이전트로 응답을 다시 보내기 위해 event queue를 관리합니다.

4. **Error Handling**  
   다양한 실패 시나리오에 대해 우아한 에러 처리를 제공합니다.

핵심 메서드는 다음과 같습니다:

- **`execute()`**: 들어오는 A2A 요청을 처리하고 내부 에이전트를 호출합니다.  
- **`cancel()`**: 취소 요청을 처리합니다(이 예제에서는 구현되지 않음).

---

### Agent Card and Server Configuration

방금 만든 설정 함수들은 다음과 같은 중요한 역할을 수행합니다:

#### **Agent Skill Definition**
- 에이전트가 수행할 수 있는 기능(`flight_booking`)을 정의합니다.
- 사람이 이해하기 쉬운 설명을 제공합니다.
- 사용 예시(example usage)를 포함합니다.
- 태그(tags)를 통해 기능을 분류합니다.

#### **Agent Card**
- 에이전트의 "명함"과 같은 역할을 합니다.
- 에이전트의 capabilities(예: streaming 지원 여부)를 명시합니다.
- 입력/출력 모드(text 기반 등)를 정의합니다.
- 다른 에이전트가 이 에이전트를 발견(discover)하고 사용할 수 있도록 정보를 제공합니다.

#### **Server Application**
- executor, task store, agent card를 하나로 결합합니다.
- A2A 통신을 위한 HTTP 엔드포인트를 생성합니다.
- A2A 프로토콜의 저수준 통신 로직을 처리합니다.

In [ ]:
# Import A2A server components
from a2a.server.apps import A2AStarletteApplication
from a2a.server.request_handlers import DefaultRequestHandler
from a2a.server.tasks import InMemoryTaskStore
from a2a.types import (
    AgentCapabilities,
    AgentCard,
    AgentSkill,
)

# Server configuration
SERVER_HOST = "0.0.0.0"
SERVER_PORT = 9999

def create_flight_booking_skill() -> AgentSkill:
    """Create and return the flight booking skill configuration."""
    return AgentSkill(
        id='flight_booking',
        name='Flight Booking',
        description='Assists users in booking flights based on their requests.',
        tags=['flight', 'booking', 'travel'],
        examples=[
            'Book a flight from New York to London next Monday.',
            'I need a flight to Paris tomorrow morning.',
        ],
    )

def create_agent_card() -> AgentCard:
    """Create and return the agent card configuration."""
    return AgentCard(
        name='Semantic Kernel Flight Booking Agent',
        description='An agent that helps users book flights using semantic kernel capabilities.',
        capabilities=AgentCapabilities(streaming=True),
        url=os.environ.get('A2A_SERVER_URL'),
        version='1.0.0',
        defaultInputModes=['text'],
        defaultOutputModes=['text'],
        skills=[create_flight_booking_skill()],
        supportsAuthenticatedExtendedCard=False,
    )

def create_flight_booking_server() -> A2AStarletteApplication:
    """Create and configure the A2A server application."""
    # Initialize request handler with the flight booking agent executor
    request_handler = DefaultRequestHandler(
        agent_executor=SemanticKernelFlightBookingAgentExecutor(),
        task_store=InMemoryTaskStore(),
    )

    # Create and return the server application
    return A2AStarletteApplication(
        agent_card=create_agent_card(),
        http_handler=request_handler,
    )

print("✅ A2A Server configuration created successfully!")


### Starting the A2A Server

우리는 서버가 노트북 실행을 막지 않도록 **백그라운드 스레드**에서 서버를 시작합니다.  
이 서버는 다음과 같은 역할을 수행합니다:

1. **포트 9999**에서 들어오는 A2A 요청을 수신한다  
2. `/.well-known/agent.json` 엔드포인트에서 **agent card**를 제공해 discovery를 지원한다  
3. **A2A protocol 메시지**를 처리하고 적절한 에이전트로 라우팅한다  
4. **데몬 스레드(daemon thread)**로 실행되어, 노트북 커널이 종료되면 함께 종료된다  

> **Note**: "address already in use" 오류가 발생한다면, 이전 실행에서 서버가 아직 실행 중이라는 의미입니다.

In [ ]:
# Function to start the A2A server in a separate thread
def start_a2a_server():
    """Start the A2A server in a separate thread."""
    server = create_flight_booking_server()
    uvicorn.run(server.build(), host=SERVER_HOST, port=SERVER_PORT)

# Start the server in a background thread
server_thread = threading.Thread(target=start_a2a_server, daemon=True)
server_thread.start()


In [ ]:
async with httpx.AsyncClient() as client:
    response = await client.get(f"http://localhost:{SERVER_PORT}/.well-known/agent.json")
    if response.status_code == 200:
        print("✅ A2A Server is running successfully!")
        agent_card = response.json()
        print(f"Agent Name: {agent_card.get('name')}")
        print(f"Agent Description: {agent_card.get('description')}")
        print(f"Agent Skills: {[skill.get('name') for skill in agent_card.get('skills', [])]}")

### Server Status Verification

좋습니다! 서버가 정상적으로 실행 중이며 요청에 응답하고 있습니다.  
`/.well-known/agent.json` 엔드포인트는 표준 A2A discovery 메커니즘으로, 다른 에이전트가 다음을 수행할 수 있게 해줍니다:

1. **Discover capabilities**  
   에이전트가 어떤 skills를 제공하는지 확인할 수 있습니다.

2. **Understand interfaces**  
   이 에이전트와 어떻게 통신해야 하는지 이해할 수 있습니다.

3. **Get examples**  
   사용에 도움이 되는 sample request들을 참고할 수 있습니다.

4. **Check compatibility**  
   프로토콜 버전과 지원 기능을 확인하여 호환성을 판단합니다.

이제 우리의 flight booking agent는 A2A protocol을 통해  
**다른 에이전트가 사용할 수 있는 상태로 완전히 준비되었습니다!**

---

### Travel Planning Agent Architecture

이제 Flight Booking Agent를 **tool**로 활용하는 오케스트레이팅 에이전트인  
Travel Planning Agent를 만들어보겠습니다.

#### **Agent Configuration:**
- **General Purpose**  
  Flight Booking Agent처럼 특정 기능에만 특화된 에이전트가 아니라,  
  다양한 여행 관련 요청을 처리하는 범용 에이전트입니다.

- **Tool Integration**  
  항공편 관련 요청을 처리하기 위해 `FlightBookingTool`에 접근할 수 있습니다.

- **Smart Routing**  
  어떤 요청은 자체적으로 처리하고,  
  어떤 요청은 flight booking tool(A2A)을 호출해야 하는지 스스로 판단합니다.

- **Context Awareness**  
  Flight Booking Agent와는 **별도의 conversation history**를 유지하며,  
  자체 context 내에서 대화를 이어갑니다.

#### **How It Decides When to Use the Flight Booking Tool:**

Travel Planning Agent는 Semantic Kernel의 **function calling** 기능을 사용합니다.  
대화 중에 항공편 관련 요청이 감지되면,  
에이전트는 자동으로 `book_flight` 함수를 호출합니다.  
이 함수 호출이 바로 A2A 통신을 트리거하여 Flight Booking Agent와 상호작용하게 됩니다.

In [ ]:
# Import A2A client components
from a2a.client import A2ACardResolver, A2AClient
from a2a.types import MessageSendParams, SendMessageRequest
from semantic_kernel.functions.kernel_function_decorator import kernel_function

# Configuration
FLIGHT_BOOKING_AGENT_URL = os.getenv("A2A_SERVER_URL", "http://localhost:9999")

class FlightBookingTool:
    """Tool for booking flights using the flight booking agent."""

    @kernel_function(
        description="Book a flight using the flight booking agent",
        name="book_flight"
    )
    async def book_flight(self, user_input: str) -> str:
        """
        Book a flight using the external flight booking agent.

        Args:
            user_input: The user's flight booking request

        Returns:
            The response from the flight booking agent
        """
        try:
            async with httpx.AsyncClient() as httpx_client:
                # Resolve the agent card from the flight booking agent
                resolver = A2ACardResolver(
                    httpx_client=httpx_client, 
                    base_url=FLIGHT_BOOKING_AGENT_URL
                )
                agent_card = await resolver.get_agent_card()

                # Create A2A client
                client = A2AClient(
                    httpx_client=httpx_client,
                    agent_card=agent_card
                )

                # Prepare the request
                request = SendMessageRequest(
                    id=str(uuid4()),
                    params=MessageSendParams(
                        message={
                            "messageId": uuid4().hex,
                            "role": "user",
                            "parts": [{"text": user_input}],
                            "contextId": "travel_booking_context",
                        }
                    )
                )

                # Send the message to the flight booking agent
                response = await client.send_message(request)
                result = response.model_dump(mode='json', exclude_none=True)

                logger.info(f"Flight booking tool response: {result}")
                return result["result"]["parts"][0]["text"]

        except Exception as e:
            logger.error(f"Error booking flight: {e}")
            return f"Sorry, I encountered an error while trying to book your flight: {str(e)}"

print("✅ Flight Booking Tool created successfully!")


In [ ]:
# Travel Planning Agent Implementation
def create_travel_agent() -> ChatCompletionAgent:
    """Create and configure the travel planning agent."""
    return ChatCompletionAgent(
        service=AzureChatCompletion(),
        name="TravelPlanner",
        instructions=(
            "You are a helpful travel planning assistant. "
            "Use the provided tools to assist users with their travel plans. "
            "When users ask about flights, use the book_flight tool to help them. "
            "You can also provide general travel advice, recommendations, and assistance "
            "with other travel-related tasks."
        ),
        plugins=[FlightBookingTool()]
    )

# Global chat history store for the travel agent
travel_chat_history_store: dict[str, ChatHistory] = {}

def get_or_create_travel_chat_history(context_id: str) -> ChatHistory:
    """Get existing chat history or create a new one for the given context."""
    chat_history = travel_chat_history_store.get(context_id)

    if chat_history is None:
        chat_history = ChatHistory(
            messages=[],
            system_message=(
                "You are a travel planning assistant. "
                "Your task is to help the user with their travel plans, including booking flights."
            )
        )
        travel_chat_history_store[context_id] = chat_history
        logger.info(f"Created new ChatHistory for context ID: {context_id}")

    return chat_history

# Initialize the travel agent
travel_planning_agent = create_travel_agent()

async def chat_with_travel_agent(user_input: str, context_id: str = "default") -> str:
    """
    Handle chat requests with the travel planning agent.

    Args:
        user_input: The user's message
        context_id: Context identifier for maintaining chat history

    Returns:
        The agent's reply
    """
    logger.info(f"Received travel chat request: {user_input} with context ID: {context_id}")

    try:
        # Get or create chat history for the context
        chat_history = get_or_create_travel_chat_history(context_id)

        # Add user input to chat history
        chat_history.messages.append(
            ChatMessageContent(role="user", content=user_input))

        # Create a new thread from the chat history
        thread = ChatHistoryAgentThread(
            chat_history=chat_history, thread_id=str(uuid4()))

        # Get response from the agent
        response = await travel_planning_agent.get_response(message=user_input, thread=thread)

        # Add assistant response to chat history
        chat_history.messages.append(ChatMessageContent(
            role="assistant", content=response.content.content))

        logger.info(f"Travel agent response: {response.content.content}")

        return response.content.content

    except Exception as e:
        logger.error(f"Error processing travel chat request: {e}")
        return "Sorry, I encountered an error processing your request. Please try again."

print("✅ Travel Planning Agent created successfully!")


### Test the A2A Communication

이제 Agent-to-Agent(A2A) 통신을 테스트해 보겠습니다!  
**Travel Planning Agent**와 대화를 시작하면, 필요한 경우 이 에이전트가 **Flight Booking Agent**를 도구로 호출하여 항공편 예약을 처리합니다.

In [ ]:
# Test 1: General travel question (should not trigger flight booking)
print("🧪 Test 1: General travel question")
response = await chat_with_travel_agent(
    "What are some good travel destinations for summer?",
    "demo_context"
)
print("-" * 50)
print("Final Response")
print("-" * 50)
print(f"Agent: {response}")
print("-" * 50)

In [ ]:
# Test 2: Flight booking request (should trigger A2A communication)
print("🧪 Test 2: Flight booking request (A2A communication)")
response = await chat_with_travel_agent(
    "I need to book a flight from San Francisco to Tokyo on December 25th",
    "demo_context"
)
print("-" * 50)
print("Final Response")
print("-" * 50)
print(f"Agent: {response}")
print("-" * 50)

In [ ]:

    # Test 3: Follow-up with more flight details (should continue A2A communication)
print("🧪 Test 3: Follow-up with more flight details")
response = await chat_with_travel_agent(
    "Make it for 2 passengers, and I prefer business class",
    "demo_context"
)
print("-" * 50)
print("Final Response")
print("-" * 50)
print(f"Agent: {response}")
print("-" * 50)

### Understanding the Test Results

A2A 통신 테스트에서 어떤 일이 일어났는지 분석해봅시다:

#### **Test 1: General Travel Question**
- **No A2A Call**  
  Travel Planning Agent가 자체적으로 처리했으며, Flight Booking Agent를 호출하지 않았습니다.
- **Direct Response**  
  일반 여행 추천을 자체 knowledge로 제공했습니다.
- **Smart Routing**  
  이것이 flight booking intent가 아닌 **일반 여행 질문**임을 정확하게 판단했습니다.

#### **Test 2: Flight Booking Request**
- **A2A Triggered**  
  Travel Planning Agent가 flight booking 의도를 감지하여 Flight Booking Tool을 호출했습니다.
- **Protocol Communication**  
  로그에서 Flight Booking Agent로 향하는 HTTP 요청을 확인할 수 있습니다.
- **Agent Card Fetching**  
  시스템이 agent의 capabilities를 파악하기 위해 agent card를 조회했습니다.
- **Message Routing**  
  요청이 올바른 형식으로 구성되어 Flight Booking Agent로 전달되었습니다.
- **Response Handling**  
  Flight Booking Agent의 응답이 Travel Planning Agent를 통해 반환되었습니다.

#### **Test 3: Follow-up Details**
- **Continued A2A**  
  추가 정보를 전달하기 위해 또다시 A2A 호출이 발생했습니다.
- **Context Preservation**  
  Flight Booking Agent는 이전 대화 context를 그대로 유지했습니다.
- **Successful Completion**  
  필요한 모든 정보가 제공되어 예약이 성공적으로 완료되었습니다.

이 테스트들은 A2A 통신의 강력함을 잘 보여줍니다.  
각 에이전트가 **전문화된 역할을 유지하면서도**, 서로 **완벽하게 협력하여 하나의 작업을 완성**할 수 있습니다!

In [ ]:
print("🧪 Test 4: Mixed request (travel advice + flight booking)")
response = await chat_with_travel_agent(
    "What's the best time to visit Paris? Also, can you book me a flight there for next month?",
    "demo_context2"
)
print("-" * 50)
print("Final Response")
print("-" * 50)
print(f"Agent: {response}")
print("-" * 50)

In [ ]:
print("🧪 Test 5: Follow up to finalize the booking)")
response = await chat_with_travel_agent(
    "It's going to be just for myself, departing from Cairo on the 15th of November for 5 days and lets do Economy class",
    "demo_context2"
)
print("-" * 50)
print("Final Response")
print("-" * 50)
print(f"Agent: {response}")
print("-" * 50)

### Advanced A2A Features Demonstrated

이번 테스트에서는 A2A의 다양한 **고급 기능**이 잘 드러났습니다:

#### **Test 4: Mixed Request Handling**
- **Hybrid Processing**  
  Travel Planning Agent가 하나의 요청 안에서 여행 조언과 항공권 예약을 함께 처리했습니다.
- **Intelligent Routing**  
  여행 조언은 직접 처리하고, 항공권 예약은 A2A를 통해 Flight Booking Agent에 위임했습니다.
- **Context Switching**  
  일반 지식 응답과 specialized tool 사용을 자연스럽게 오가며 처리했습니다.

#### **Test 5: Multi-Turn A2A Conversations**
- **Stateful A2A**  
  Flight Booking Agent는 여러 번의 A2A 호출 간에 context를 유지했습니다.
- **Progressive Information Gathering**  
  각 A2A 요청이 이전 상호작용을 기반으로 점진적으로 정보를 축적했습니다.
- **Error Handling**  
  불완전하거나 충돌되는 정보에도 시스템이 안정적으로 대응했습니다.

#### **Key Observations:**
1. **Automatic Tool Selection**  
   Travel Planning Agent가 A2A를 언제 사용해야 하는지 스스로 판단합니다.
2. **Context Preservation**  
   두 에이전트는 각각 독립적인 conversation context를 유지합니다.
3. **Protocol Transparency**  
   A2A 통신은 최종 사용자에게 완전히 투명하며, 사용자 경험을 해치지 않습니다.
4. **Error Resilience**  
   네트워크 문제나 agent 부재 같은 상황에서도 시스템은 우아하게 대응합니다.

### Interactive Chat

이제 Travel Planning Agent와 직접 상호작용할 수 있습니다.  
여행지 추천, 항공권 예약, 또는 여행 관련 어떤 질문이든 자유롭게 해보세요:

In [ ]:
# Interactive Chat Function
async def interactive_chat():
    """Interactive chat with the Travel Planning Agent."""
    print("💬 Starting interactive chat with Travel Planning Agent")
    print("Type 'exit' to end the conversation")
    print("-" * 50)
    
    context_id = "interactive_session"
    
    while True:
        try:
            # In a real Jupyter environment, you might want to use input() or ipywidgets
            user_input = input("You: ")
            
            if user_input.lower() in ['exit', 'quit', 'bye']:
                print("👋 Thank you for chatting! Goodbye!")
                break
                
            if user_input.strip():
                response = await chat_with_travel_agent(user_input, context_id)
                print(f"Travel Agent: {response}")
                print("-" * 50)
                
        except KeyboardInterrupt:
            print("\n👋 Chat ended by user. Goodbye!")
            break
        except Exception as e:
            print(f"❌ Error: {e}")
            print("-" * 50)

# Example usage (uncomment to run interactively)
# await interactive_chat()

# For demonstration, let's show some example interactions
print("💡 Example interactions you can try:")
print("- 'What are the best beaches in Thailand?'")
print("- 'Book a flight from London to New York tomorrow'")
print("- 'What's the weather like in Tokyo in spring?'")
print("- 'I need a round-trip flight to Paris for 2 people'")
print("- 'Tell me about travel insurance options'")
print("")
print("💬 To start interactive chat, uncomment and run: await interactive_chat()")


### A2A Communication Flow Diagram

여행 에이전트가 항공권 예약이 필요하다고 판단했을 때의 흐름은 다음과 같습니다:

```
┌─────────────────┐    1. User Request   ┌─────────────────┐
│                 │ ───────────────────► │                 │
│      User       │                      │ Travel Planning │
│                 │ ◄─────────────────── │     Agent       │
└─────────────────┘    6. Final Response └─────────────────┘
                                                   │
                                                   │ 2. Detects Flight
                                                   │    Booking Intent
                                                   ▼
                                         ┌─────────────────┐
                                         │                 │
                                         │  FlightBooking  │
                                         │     Tool        │
                                         │                 │
                                         └─────────────────┘
                                                   │
                                                   │ 3. A2A Protocol
                                                   │    Communication
                                                   ▼
                                         ┌─────────────────┐
                                         │                 │
                                         │ Flight Booking  │
                                         │     Agent       │
                                         │   (A2A Server)  │
                                         │                 │
                                         └─────────────────┘
                                                   │
                                                   │ 4. Processes Request
                                                   │ 5. Returns Response
                                                   ▼
```


이 아키텍처는 다음과 같은 이점을 제공합니다:

- **Separation of Concerns**  
  각 에이전트가 자신의 역할에만 집중할 수 있습니다.

- **Reusability**  
  Flight Booking Agent는 여러 Travel Planning Agent에서 재사용할 수 있습니다.

- **Scalability**  
  각 에이전트는 독립적으로 배포 및 스케일링할 수 있습니다.

- **Maintainability**  
  항공권 예약 로직을 수정해도 Travel Planning Agent 로직에는 영향을 주지 않습니다.



## Summary and Key Concepts

축하합니다! 이제 Semantic Kernel을 사용한 Agent-to-Agent(A2A) 통신을 완전히 구현해냈습니다.  
아래는 우리가 이 노트북에서 구축한 것과 핵심 개념 요약입니다.

---

### 🎯 What We Built

1. **Flight Booking Agent**  
   항공권 예약만 담당하는 전문 에이전트

2. **A2A Server**  
   Flight Booking Agent를 HTTP API 형태로 외부에 노출하는 서버

3. **Travel Planning Agent**  
   여행 전반을 도와주는 에이전트로, 필요할 때 Flight Booking Agent를 도구처럼 사용

4. **A2A Communication**  
   두 에이전트 간의 자연스럽고 투명한 통신 흐름 구현

---

### 🔑 Key Concepts Learned

#### 1. **Agent Specialization**
- 각 에이전트는 분명한 역할과 능력을 가짐
- 역할 특화는 성능, 확장성, 유지보수성 향상으로 이어짐
- 서로 독립적으로 개발 및 배포가 가능

#### 2. **A2A Protocol**
- 에이전트 간 통신을 위한 표준화된 프로토콜
- Agent Card, Skill Definition, 메시지 포맷이 포함됨
- 다른 에이전트가 기능을 “발견(discovery)”하고 사용할 수 있게 해줌

#### 3. **Tool Integration**
- 한 에이전트가 다른 에이전트를 “툴(tool)”로 호출 가능
- Semantic Kernel의 Function Calling이 A2A에도 동일하게 적용됨
- 복잡한 오케스트레이션과 워크플로 구성이 가능해짐

#### 4. **Context Management**
- 각 에이전트는 자신의 대화 맥락(Context)을 독립적으로 유지
- Context ID를 통해 여러 사용자 또는 여러 세션을 구분
- 에이전트 간 경계를 넘는 stateful 대화가 가능

---

### 🏗️ Architecture Benefits

1. **Modularity**  
   에이전트별로 개발, 테스트, 배포가 완전히 독립적

2. **Scalability**  
   Flight Booking Agent만 트래픽이 많으면 그것만 수평 확장 가능

3. **Reusability**  
   하나의 Flight Booking Agent를 여러 Travel Agent가 재사용 가능

4. **Maintainability**  
   담당 역할이 분리되어 있어 유지보수가 훨씬 용이

---

### 🚀 Next Steps

이제 다음과 같은 확장을 시도해볼 수 있습니다:

1. **추가 전문 에이전트 생성**  
   호텔 예약, 렌터카 예약, 맛집 추천 등

2. **실제 API 연동**  
   GDS(Global Distribution System), 호텔/항공사 API 등과 연결해 실제 예약 시스템 구현

---

이 튜토리얼은 에이전트 간 통신(A2A)의 강력함을 잘 보여줍니다.  
모듈형 구조를 통해 복잡한 분산 AI 시스템도 깔끔하게 구성하고 유지할 수 있습니다.

## Optional: Cleanup

백그라운드에서 실행 중인 A2A 서버를 종료하고 싶다면, 아래 코드를 실행하면 됩니다:

In [ ]:
# Optional: Cleanup
# Note: The server thread is running as a daemon thread, so it will automatically
# terminate when the main Python process ends. If you want to explicitly check
# if the server is still running, you can use:

if server_thread.is_alive():
    print("✅ A2A Server is still running in the background")
    print("🔧 The server will automatically stop when the notebook kernel is restarted")
else:
    print("❌ A2A Server thread has stopped")

# You can also check the server status again
print("\n🔍 Final server status check:")
try:
    async with httpx.AsyncClient() as client:
        response = await client.get(f"http://localhost:{SERVER_PORT}/.well-known/agent.json")
        if response.status_code == 200:
            print("✅ A2A Server is still responding")
        else:
            print(f"❌ A2A Server returned status code: {response.status_code}")
except Exception as e:
    print(f"❌ Cannot connect to A2A Server: {e}")

print("\n🎉 Tutorial completed successfully!")
print("💡 Remember: The A2A server runs as a daemon thread and will stop automatically when the notebook kernel is restarted.")


## Running the Standalone Agent Files

이 튜토리얼에는 두 에이전트를 **독립 실행(standalone)** 형태로 구동할 수 있는 구현이 포함되어 있습니다:

---

### Flight Booking Agent Server  
`flight-booking-agent/` 디렉터리 안에 **Flight Booking Agent**를 standalone 서버로 실행할 수 있습니다:

```bash
cd flight-booking-agent/
uv run server.py
```

이 명령을 실행하면 `http://localhost:9999` 에서 A2A 서버가 실행되며, Flight Booking Agent가 준비됩니다.

### Travel Booking Agent
웹 인터페이스를 포함한 Travel Planning Agent는 `travel-booking-agent/` 디렉터리에 있습니다:

```bash
cd travel-booking-agent/
uv run agent.py
```

이 명령을 실행하면 로컬 웹 서버가 실행되며, 브라우저를 통해 Travel Planning Agent와 상호작용할 수 있습니다.